In [1]:
import cv2
import mediapipe as mp

# Initialize the MediaPipe Pose and Drawing utils
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize BlazePose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Capture video from webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Failed to grab frame")
        break

    # Convert the frame to RGB for MediaPipe Pose
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False

    # Run BlazePose detection
    results = pose.process(image_rgb)

    # Convert back to BGR for OpenCV
    image_rgb.flags.writeable = True
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    # Draw the pose landmarks on the frame
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image_bgr, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),  # For landmarks
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)  # For connections
        )

    # Show the frame with BlazePose landmarks
    cv2.imshow('BlazePose on Webcam', image_bgr)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [4]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe BlazePose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Capture video from webcam
cap = cv2.VideoCapture(0)

# Variables to track push-up count and state
pushup_count = 0
is_down_position = False  # Indicates if the user is in the "down" part of the push-up

def calculate_angle(a, b, c):
    """Calculates the angle between three points (in degrees)."""
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360.0 - angle

    return angle

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert frame to RGB for MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False

    # Run BlazePose
    results = pose.process(image_rgb)

    # Convert back to BGR for OpenCV
    image_rgb.flags.writeable = True
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    # If landmarks are detected
    if results.pose_landmarks:
        # Draw pose landmarks on the image
        mp_drawing.draw_landmarks(image_bgr, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Get landmark positions
        landmarks = results.pose_landmarks.landmark

        # Extract keypoints for shoulder, elbow, and wrist
        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

        # Calculate the angle at the left elbow
        elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

        # Detect the push-up movement
        if elbow_angle < 90:  # The "down" position (arms bent)
            is_down_position = True
        elif elbow_angle > 160 and is_down_position:  # The "up" position (arms straight)
            pushup_count += 1
            is_down_position = False

        # Display the count on the image
        cv2.putText(image_bgr, f'Push-ups: {pushup_count}', (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Show the image
    cv2.imshow('Push-up Counter', image_bgr)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import json

# Initialize MediaPipe BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Capture video from webcam
cap = cv2.VideoCapture(0)

# Store keypoints over time
keypoints_list = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Convert frame to RGB for MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False

    # Run BlazePose
    results = pose.process(image_rgb)

    # Convert back to BGR for OpenCV
    image_rgb.flags.writeable = True
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

    # If landmarks are detected
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        
        # Extract and store the keypoints (x, y, z) and visibility for each joint
        keypoints = []
        for landmark in landmarks:
            keypoints.append((landmark.x, landmark.y, landmark.z, landmark.visibility))
        
        keypoints_list.append(keypoints)

        # Draw the pose landmarks on the frame (for visualization)
        mp.solutions.drawing_utils.draw_landmarks(
            image_bgr, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS
        )

    # Show the frame
    cv2.imshow('Skeleton Capture', image_bgr)

    # Press q to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# Save keypoints_list to a JSON file
with open('skeleton_data.json', 'w') as f:
    json.dump(keypoints_list, f)

cap.release()
cv2.destroyAllWindows()

C:\Users\dougl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [2]:
import cv2
import json
import numpy as np

# Define keypoint connections (from BlazePose)
keypoint_connections = [
    (6,5),
    (4,5),
    (6,8),

    (1,2),
    (2,3),
    (3,7),

    (10,9),

    (20,18),
    (18,16),
    (16,22),
    (16,20),

    (16,14),
    (14,12),
    (12,11),
    (11,13),
    (13,15),
    (15,21),
    (15,19),
    (15,17),
    (19,17),

    (12,24),
    (11,23),
    (24,23),

    (24,26),
    (26,28),
    (32,28),
    (30,28),
    (32,30),
    
    (23,25),
    (27,25),
    (27,29),
    (29,31),
    (27,31)
]

# Load keypoints from JSON
with open('skeleton_data.json', 'r') as f:
    loaded_keypoints_list = json.load(f)

# Create a window to replay the movements
window_name = 'Skeleton Replay'
cv2.namedWindow(window_name)

# Replay the keypoints data
for keypoints in loaded_keypoints_list:
    # Create a blank frame
    frame = np.zeros((480, 640, 3), dtype=np.uint8)
    
    # Scale the keypoints to fit the window size (assuming original BlazePose keypoints are normalized between 0 and 1)
    height, width = frame.shape[:2]
    scaled_keypoints = []
    for i, (x, y, z, visibility) in enumerate(keypoints):
        if visibility > 0.4:  # Only show keypoints with good visibility
            cx, cy = int(x * width), int(y * height)
            scaled_keypoints.append((cx, cy))  # Store for drawing lines
            # Draw keypoint as a small circle
            cv2.circle(frame, (cx, cy), 5, (255, 255, 255), -1)
        else:
            scaled_keypoints.append(None)  # If not visible, mark as None
    
    # Draw the skeleton by connecting keypoints
    for start_idx, end_idx in keypoint_connections:
        if scaled_keypoints[start_idx] is not None and scaled_keypoints[end_idx] is not None:
            cv2.line(frame, scaled_keypoints[start_idx], scaled_keypoints[end_idx], (0, 255, 0), 2)
    
    # Show the frame
    cv2.imshow(window_name, frame)

    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [ ]:
import cv2
import mediapipe as mp
import json

# Initialize MediaPipe BlazePose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Load video file
video_path = 'workout.mp4'  # Change to your video path
cap = cv2.VideoCapture(video_path)

# Store keypoints over time
keypoints_list = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Video has ended.")
        break

    # Convert frame to RGB for MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image_rgb.flags.writeable = False

    # Run BlazePose
    results = pose.process(image_rgb)

    # If landmarks are detected
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Extract and store the keypoints (x, y, z) and visibility for each joint
        keypoints = []
        for landmark in landmarks:
            keypoints.append((landmark.x, landmark.y, landmark.z, landmark.visibility))
        
        keypoints_list.append(keypoints)

    # Press 'q' to exit early
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save keypoints_list to a JSON file
with open('video_skeleton_data.json', 'w') as f:
    json.dump(keypoints_list, f)

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import json
import numpy as np

# Define keypoint connections (from BlazePose)
keypoint_connections = [
    (6,5),
    (4,5),
    (6,8),

    (1,2),
    (2,3),
    (3,7),

    (10,9),

    (20,18),
    (18,16),
    (16,22),
    (16,20),

    (16,14),
    (14,12),
    (12,11),
    (11,13),
    (13,15),
    (15,21),
    (15,19),
    (15,17),
    (19,17),

    (12,24),
    (11,23),
    (24,23),

    (24,26),
    (26,28),
    (32,28),
    (30,28),
    (32,30),
    
    (23,25),
    (27,25),
    (27,29),
    (29,31),
    (27,31)
]

# Load keypoints from the previously saved JSON file
with open('video_skeleton_data.json', 'r') as f:
    loaded_keypoints_list = json.load(f)

# Load video again for playback
video_path = 'your_video.mp4'  # Change to your video path
cap = cv2.VideoCapture(video_path)

# Replay skeleton over video frames
for keypoints in loaded_keypoints_list:
    ret, frame = cap.read()
    if not ret:
        print("Video has ended.")
        break

    height, width = frame.shape[:2]
    
    # Scale the keypoints to fit the video frame size
    scaled_keypoints = []
    for i, (x, y, z, visibility) in enumerate(keypoints):
        if visibility > 0.5:  # Only show keypoints with good visibility
            cx, cy = int(x * width), int(y * height)
            scaled_keypoints.append((cx, cy))  # Store for drawing lines
            # Draw keypoint as a small circle
            cv2.circle(frame, (cx, cy), 5, (255, 255, 255), -1)
        else:
            scaled_keypoints.append(None)  # If not visible, mark as None
    
    # Draw the skeleton by connecting keypoints
    for start_idx, end_idx in keypoint_connections:
        if scaled_keypoints[start_idx] is not None and scaled_keypoints[end_idx] is not None:
            cv2.line(frame, scaled_keypoints[start_idx], scaled_keypoints[end_idx], (0, 255, 0), 2)
    
    # Display the frame with the skeleton
    cv2.imshow('Skeleton Replay', frame)

    # Press 'q' to exit early
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from mediapipe import solutions

# Initialize BlazePose
pose = solutions.pose.Pose()

def calculate_angle(a, b, c):
    a = np.array(a)  # Shoulder
    b = np.array(b)  # Elbow
    c = np.array(c)  # Wrist
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

cap = cv2.VideoCapture(0)
count = 0
stage = None

while cap.isOpened():
    ret, frame = cap.read()
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    
    # Get the frame's height and width for pixel conversion
    height, width, _ = frame.shape
    
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        
        # Get coordinates for key landmarks
        shoulder_left = [landmarks[11].x * width, landmarks[11].y * height]
        elbow_left = [landmarks[13].x * width, landmarks[13].y * height]
        wrist_left = [landmarks[15].x * width, landmarks[15].y * height]
        
        # Calculate the elbow angle
        elbow_angle = calculate_angle(shoulder_left, elbow_left, wrist_left)
        
        # Push-up counter logic
        if elbow_angle > 160:
            stage = "up"
        if elbow_angle < 90 and stage == "up":
            stage = "down"
            count += 1
            print(f"Push-up count: {count}")
        
        # Find the min and max coordinates to create a bounding box
        x_values = [landmark.x for landmark in landmarks]
        y_values = [landmark.y for landmark in landmarks]
        
        # Convert normalized coordinates to pixel values
        x_min = int(min(x_values) * width)
        x_max = int(max(x_values) * width)
        y_min = int(min(y_values) * height)
        y_max = int(max(y_values) * height)
        
        # Draw the bounding box around the person
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    
    # Display the frame with the bounding box and push-up counter
    cv2.putText(frame, f'Push-Up Count: {count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.imshow('Push-Up Counter', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()